### Study of the instrinsic energy imbalance in IFS

Appunti da EC-Earth3 sources

**in src/ifs/phys_ec/vdfmain.F90**
- !    *PAHFLEV*      LATENT HEAT FLUX  (SNOW/ICE FREE PART)        W/M2
- !    *PAHFLSB*      LATENT HEAT FLUX  (SNOW/ICE COVERED PART)     W/M2

**in src/ifs/phys_ec/vdfouter.F90**
- (initialize) PAHFLSB(KIDIA:KFDIA)=0.0_JPRB
- (call) CALL VDFMAIN ( ... ZAHFLEV, **ZAHFLSB**, ...  )
   
-  (accumulate fluxes) PAHFLSB(KIDIA:KFDIA)=PAHFLSB(KIDIA:KFDIA)+ZAHFLSB(KIDIA:KFDIA)
- (compute average fluxes) PAHFLSB(KIDIA:KFDIA) =PAHFLSB(KIDIA:KFDIA) *ZINVDF

**in src/ifs/phys_ec/vdfmain.F90**

- !**   *VDFMAIN* - DOES THE VERTICAL EXCHANGE OF U,V,SLG,QT BY TURBULENCE.
- CALL SURFPP(... PAHFLEV=PAHFLEV, **PAHFLSB=PAHFLSB**, ...)


**in src/surf/external/surfpp.F90**

IF(UBOUND(PAHFLSB,1) < KLON) THEN
  CALL ABORT_SURF('SURFPP: PAHFLSB TOO SHORT!')
ENDIF

- CALL SURFPP_CTL

!     ** SURFPP_CTL CALLS SUCCESSIVELY:
- !         *SPPCFL* !**   *SPPCFL* - COMPUTES THE SURFACE (2 M) TEMPERATURE AND HUMIDITY WITH STABILITY AS FUNCTION OF OBUKHOV-L
- !         *SPPGUST* !**   *SPPGUST* - COMPUTES THE area averaged 10 m wind and the gust
- !         *VOSKIN* !**   *VOSKIN* - COMPUTES WARM AND COLD SKIN EFFECTS OVER THE OCEAN

**in src/surf/module/surfpp_ctl_mod.F90**

- !      PAHFLSB   :  LATENT HEAT FLUX  (SNOW/ICE COVERED PART)    
- !      PAHFLTI   :  Surface latent heat flux   
- !      PFWSB     :  EVAPORATION OF SNOW                              kg/(m**2*s)
- !      PEVAPSNW :    Evaporation from snow under forest  kg m-2 s-1
- !      PFRTI     :  TILE FRACTIONS
  - 1 : WATER                  
  - 2 : ICE                    
  - 3 : WET SKIN               
  - 4 : DRY SNOW-FREE LOW-VEG  
  - 5 : SNOW ON LOW-VEG+BARE-SOIL
  - 6 : DRY SNOW-FREE HIGH-VEG
  - 7 : SNOW UNDER HIGH-VEG
  - 8 : BARE SOIL

USE YOS_CST   ,ONLY : RLSTT, RD, RETV
- REAL(KIND=JPRB) :: RLSTT   ! Thermodynamic transition of phase
- REAL(KIND=JPRB) :: RLMLT   ! Thermodynamic transition of phase
- REAL(KIND=JPRB) :: RLVTT   ! Thermodynamic transition of phase

!*         1.2  PARAMETERS AND DERIVATIVES (SET TO 0) FOR LAND 
!*              SURFACE SCHEME

DO JL=KIDIA,KFDIA
  PAHFLEV(JL)=PFRTI(JL,1)*PAHFLTI(JL,1)&
   & +PFRTI(JL,3)*PAHFLTI(JL,3)&
   & +PFRTI(JL,4)*PAHFLTI(JL,4)&
   & +PFRTI(JL,6)*PAHFLTI(JL,6)&
   & +PFRTI(JL,7)*(PAHFLTI(JL,7)-RLSTT*PEVAPSNW(JL))&
   & +PFRTI(JL,8)*PAHFLTI(JL,8) 
  IF (LEFLAKE) THEN
    PAHFLEV(JL)=PAHFLEV(JL)+PFRTI(JL,9)*PAHFLTI(JL,9)     
  ENDIF
  
  PAHFLSB(JL)=PFRTI(JL,2) * PAHFLTI(JL,2)+PFRTI(JL,5)*PAHFLTI(JL,5)+PFRTI(JL,7) * RLSTT * PEVAPSNW(JL)
  
  PFWSB(JL) = PFRTI(JL,5)*PEVAPTI(JL,5)&
   & +PFRTI(JL,7)*PEVAPSNW(JL)  
ENDDO








**in src/surf/module/voskin_mod.F90**

- !     *PSSRFL*       NET SOLAR RADIATION AT THE SURFACE
- !     *PSLRFL*       NET THERMAL RADIATION AT THE SURFACE
- !     *PAHFS*        SURFACE SENSIBLE HEAT FLUX
- !     *PAHFL*        SURFACE LATENT HEAT FLUX

- !     Atmospheric buoyancy and wind
    ZROWQ=PAHFL(JL)/RLVTT

- !     2.2 Warm layer; formulation C (Xubin Zeng)
!
    DO JL=KIDIA,KFDIA
        ZDSST=PTSKM1M(JL)-PSST(JL)-ZDCOOL(JL)

        ZSRD=(PSSRFL(JL)*ZFI+PSLRFL(JL)+PAHFS(JL)+PAHFL(JL))/ZROC

!     3. Cool skin (Fairall et al. 1996)

IF (LEOCCO) then
  DO JL=KIDIA,KFDIA

- !      3.2 Apply empirical formulas

    ZUSTW2=ZROADRW*ZUST(JL)**2
    ZQ=MAX(1.0_JPRB,-PSLRFL(JL)-PAHFS(JL)-PAHFL(JL))


**in src/surf/module/sppcfl_mod.F90**










